In [ ]:

# Task 0: Imports and setup
import pandas as pd  # pandas for DataFrame operations
import numpy as np   # numpy for vectorized numeric ops
from pathlib import Path  # Path for file paths
base = Path('.')  # use current directory for reading/writing CSVs


In [ ]:

# Task 1: Verify and use the three saved CSV files to create DataFrames
employee_df = pd.read_csv(base / 'employee_df.csv')  # load employee data from CSV
seniority_df = pd.read_csv(base / 'seniority_df.csv')  # load seniority data from CSV
project_df = pd.read_csv(base / 'project_df.csv')  # load project data from CSV
print('Employee:'); print(employee_df)  # display Employee
print('Seniority:'); print(seniority_df)  # display Seniority
print('Project:'); print(project_df)  # display Project

In [ ]:

# Task 2: Fill missing 'Cost' values in Project using running average and a for-loop
project_df['Cost'] = pd.to_numeric(project_df['Cost'], errors='coerce')  # convert Cost to numeric, NaN for blanks
running_sum = 0.0  # initialize running sum of known costs
count = 0         # initialize count of known costs
for i in range(len(project_df)):  # iterate row indices
    val = project_df.loc[i, 'Cost']  # read current cost value
    if pd.isna(val):  # if current cost is missing
        if count > 0:  # if we have at least one prior known value
            project_df.loc[i, 'Cost'] = running_sum / count  # set to running average
        else:  # edge-case: first value missing (not present in this dataset)
            project_df.loc[i, 'Cost'] = project_df['Cost'].mean(skipna=True)  # fallback to global mean
    else:  # if current cost is known
        running_sum += val  # add to running sum
        count += 1          # increment count
project_df.to_csv(base / 'project_df.csv', index=False)  # save updated Project CSV in-place
print('Updated Project (Task 2):'); print(project_df)  # show result


In [ ]:

# Task 3: Split Employee 'Name' into 'First Name' and 'Last Name', then drop 'Name'
employee_df[['First Name', 'Last Name']] = employee_df['Name'].str.split(' ', 1, expand=True)  # split once
employee_df = employee_df.drop(columns=['Name'])  # remove the original Name column
employee_df.to_csv(base / 'employee_df.csv', index=False)  # save updated Employee CSV
print('Updated Employee (Task 3):'); print(employee_df)  # show result


In [ ]:

# Task 4: Join Employee + Seniority + Project into a single DataFrame 'Final'
employee_df = pd.read_csv(base / 'employee_df.csv')  # re-load updated Employee
seniority_df = pd.read_csv(base / 'seniority_df.csv')  # load Seniority
project_df = pd.read_csv(base / 'project_df.csv')  # load updated Project
Final = employee_df.merge(seniority_df, on='ID', how='left')  # join Employee with Seniority
Final = Final.merge(project_df, on='ID', how='left')  # join with Project (one-to-many)
print('Final (Task 4):'); print(Final)  # show result


In [ ]:

# Task 5: Add 'Bonus' column: 5% of cost for projects with Status == 'Finished'
Final['Bonus'] = np.where(Final['Status'].str.lower() == 'finished', Final['Cost'] * 0.05, 0.0)  # compute bonus
print('Final (Task 5 - Bonus added):'); print(Final)  # show result


In [ ]:

# Task 6: Demote by 1 for employees whose projects have Status 'Failed'; remove records with designation > 4
failed_ids = Final.loc[Final['Status'].str.lower() == 'failed', 'ID'].unique()  # IDs with any failed project
Final['Designation Level'] = Final['Designation Level'].astype(int)  # ensure integer type
Final.loc[Final['ID'].isin(failed_ids), 'Designation Level'] = Final.loc[Final['ID'].isin(failed_ids), 'Designation Level'] + 1  # demote by +1 level
Final = Final[Final['Designation Level'] <= 4]  # delete records where designation level > 4
print('Final (Task 6 - Demotions applied & >4 removed):'); print(Final)  # show result


In [ ]:

# Task 7: Prefix 'Mr.' or 'Mrs.' to 'First Name' based on original Gender and drop the 'Gender' column
Final['First Name'] = np.where(Final['Gender'].str.upper() == 'M', 'Mr. ' + Final['First Name'], 'Mrs. ' + Final['First Name'])  # prefix
Final = Final.drop(columns=['Gender'])  # drop Gender column
print('Final (Task 7 - Titles applied & Gender dropped):'); print(Final)  # show result


In [ ]:

# Task 8: Promote designation level by 1 for employees with Age > 29 (cap minimum at 1)
Final['Designation Level'] = np.where(Final['Age'] > 29, Final['Designation Level'] - 1, Final['Designation Level'])  # promote by -1
Final['Designation Level'] = Final['Designation Level'].clip(lower=1)  # ensure not below 1
print('Final (Task 8 - Promotions applied):'); print(Final)  # show result


In [ ]:

# Task 9: Sum the 'Cost' of all projects per employee; save as 'TotalProjCost' with ID, First Name, Total cost
TotalProjCost = Final.groupby(['ID', 'First Name'], as_index=False)['Cost'].sum().rename(columns={'Cost': 'Total cost'})  # group & sum
TotalProjCost.to_csv(base / 'TotalProjCost.csv', index=False)  # save to CSV
print('TotalProjCost (Task 9):'); print(TotalProjCost)  # show result


In [ ]:

# Task 10: Print employee details whose City contains the letter 'o' (case-insensitive)
contains_o = Final[Final['City'].str.contains('o', case=False, na=False)]  # filter rows
print('Employees with City containing "o" (Task 10):'); print(contains_o)  # show result
